### Checklist for submission

It is extremely important to make sure that:

1. Everything runs as expected (no bugs when running cells);
2. The output from each cell corresponds to its code (don't change any cell's contents without rerunning it afterwards);
3. All outputs are present (don't delete any of the outputs);
4. Fill in all the places that say `YOUR CODE HERE`, or "**Your answer:** (fill in here)".
5. You **ONLY** change the parts of the code we asked you to, nowhere else (change only the coding parts saying `# YOUR CODE HERE`, nothing else);
6. Don't add any new cells to this notebook;
7. Fill in your group number and the full names of the members in the cell below;
8. Make sure that you are not running an old version of IPython (we provide you with a cell that checks this, make sure you can run it without errors).

Failing to meet any of these requirements might lead to either a subtraction of POEs (at best) or a request for resubmission (at worst).

We advise you the following steps before submission for ensuring that requirements 1, 2, and 3 are always met: **Restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). This might require a bit of time, so plan ahead for this (and possibly use Google Cloud's GPU in HA1 and HA2 for this step). Finally press the "Save and Checkout" button before handing in, to make sure that all your changes are saved to this .ipynb file.

---

Group number and member names:

In [2]:
GROUP = "19"
NAME1 = "Kamila Kowalska"
NAME2 = "Sandra Viknander"

Make sure you can run the following cell without errors.

In [1]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

---

# HA1 - Cats and dogs

<img src="http://lghttp.32478.nexcesscdn.net/80E972/organiclifestylemagazine/wp-content/uploads/2015/10/Cats-and-Dogs.jpg" alt="Cats and dogs" style="width: 5000px;"/>

For this home assignment, we'll use the Kaggle dataset for the [Dogs vs. Cats competition](https://www.kaggle.com/c/dogs-vs-cats). It is comprised of 25k colored images of dogs and cats. Our goal with this dataset will be to create a classifier that can tell us if the input image is of a cat or a dog.

As a way of helping you speed up the training process, each group gets 6 hours of access to an instance in Google Cloud with a K80 GPU. Take a look at the [Instructions folder](https://github.com/JulianoLagana/deep-machine-learning/tree/master/Instructions) to understand how to connect to this instance and use our tools there. You're free to use this resource as you see fit, but if you run out of hours you'll need a late day to obtain more (and you can only do this once).

In order to make the most out of your GPU hours, first try solving the initial part of this notebook (tasks 0-4) in your own computer (these tasks can be solved only on the CPU), and leave most of the available hours for solving tasks 5-6, and refining your best model further (and, if you have the spare hours, experiment a bit!).

Requirements:
- Whenever we ask you to plot anything, be sure to add a title and label the axes. If you're plotting more than one curve in the same plot, also add a legend.
- When we ask you to train an architecture, train it for a reasonable number of epochs. "Reasonable" here means you should be fairly confident that training for a higher number of epochs wouldn't impact your conclusions regarding the model's performance.

Tips:
- If you get errors saying you've exhausted the GPU resources, well, then you exhausted the GPU resources ;). However, sometimes that's because TensorFlow didn't release a part of the GPU's memory. If you think your CNN should fit in your memory during training, try restarting the kernel and directly training only that architecture.
- Every group has enough credits on google cloud to complete this assignment. However, this statement assumes you'll use your resources judiciously (e.g. always try the code first in your machine and make sure everything works properly before starting your instances) and **won't forget to stop your instance after using it,**  otherwise you might run out of credits.
- Before starting, take a look at the images we'll be using. This is a hard task, don't get discouraged if your first models perform poorly (several participants in the original competition didn't achieve an accuracy higher than 60%).

---
## 0. Imports

In the following cell, add all the imports you'll use in this assignment.

In [2]:
import pandas as pd
import numpy as np

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.core import Flatten
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_crossentropy

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

#from torch.utils.data import Dataset, DataLoader
#from torchvision import transforms, utils

import os
import pickle
import matplotlib.pyplot as plt

G:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


---
## 1. Loading the data and preprocessing

The first step is to head to the [Kaggle website for the cats and dogs competition](https://www.kaggle.com/c/dogs-vs-cats) and download the data from there. You should download both the test and train folders together in one zip file (by clicking the download all button). The split ratio between training and validation has not been made, you'll need to do it yourself. The `test.zip` file contains unlabeled data, so that participants in the contest are not able to train on this set.

For this assignment you should use [data generators](https://keras.io/preprocessing/image/) to load the images to your CPU/GPU memory. Because of this, your folder structure for the data should conform to the folder structure expected by the data generators (i.e. the samples should be separated into one folder for each class). Furthermore, we ask you to first start with a smaller subset of the data (1/5 of the number of samples), in order to test different models faster.

This means that you should create a folder structure that resembles the following (obviously, the folder names are up to you):


         small_train             small_val                train                   val
              |                      |                      |                      |
              |                      |                      |                      |
        -------------          -------------          -------------          -------------
        |           |          |           |          |           |          |           |
        |           |          |           |          |           |          |           |
      Cats        Dogs       Cats        Dogs       Cats        Dogs       Cats        Dogs

The `small_train` and `small_val` folders have the training and validation samples for your smaller subset of the data, while the `train` and `val` folders contain all the samples you extracted from Kaggle's `train.zip`. We provide you a notebook that shows how to achieve this ("Create project structure.ipynb"), starting from the original `all.zip` file that you download from Kaggle. If you do use that notebook, we encourage you to understand how each step is being done, so you can generalize this knowledge to new datasets you'll encounter.

We advise you to use 30% of the data as validation data in the smaller dataset. However, for the larger dataset, you should decide how to split between training and validation. Please specify your splits in the following cells.

For the larger subset, what was the training/validation split that you decided to use?

**% Samples in the training set:**

In [7]:
#**% Samples in the training set: (1750 cats; 1750 dogs) **

#**% Samples in the validation set: (750 cats; 750 dogs) **

**% Samples in the validation set:**

In [8]:
# Samples in the training set: (10000 cats; 10000 dogs)

# Samples in the validation set: (2500 cats; 2500 dogs)

Fill in the dataset paths (to be used later by your data generators):

In [4]:
train_path = 'small_train'
validation_path = 'small_val'
test_path = 'test'

---
Once you have the expected folder structure, create two data generators for automatically generating batches from the images in your smaller subset of data. Don't use any [data augmentation](https://cartesianfaith.com/2016/10/06/what-you-need-to-know-about-data-augmentation-for-machine-learning/), but feel free to preprocess the data as you see fit. After instantiating them, run the `flow_from_directory` method with the desired arguments.

Hints:
- The specified `batch_size` should be chosen so that your don't run out of memory.
- When feeding the images to your CNN, you'll probably want all of them to have the same spatial size, even though the .jpeg files differ in this. If so, take a look at the argument `target_size` for the `flow_from_directory` method of data generators.
- Resizing the images to a smaller size while loading them can be beneficial.

In [5]:
#Batch size
batch_s = 32

#picture resolution/size
img_hight = 128
img_width = 128
img_size = (img_hight, img_width)

train_batches = keras.preprocessing.image.ImageDataGenerator().flow_from_directory(train_path,
                                                                                   target_size = img_size,
                                                                                   classes=['cats','dogs'], 
                                                                                   batch_size = batch_s)

validation_batches = keras.preprocessing.image.ImageDataGenerator().flow_from_directory(validation_path,
                                                                                        target_size = img_size,
                                                                                        classes=['cats','dogs'],
                                                                                        batch_size = batch_s)

test_batches = keras.preprocessing.image.ImageDataGenerator().flow_from_directory(test_path,
                                                                                  target_size = img_size,
                                                                                  batch_size=batch_s)

Found 3500 images belonging to 2 classes.
Found 1500 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


---
## 2. Training

Create your first CNN architecture for this task. Start with something as simple as possible, that you're almost sure can get an accuracy better than 50% (we'll improve upon it later).

Tip:
- If Tensorflow is your backend, your `input_shape` is always `(img_width, img_height, 3)` (i.e. channels **last**)

In [5]:
#convolutional layers
conv_layer_1 = Conv2D(20,(7,7), activation='relu', input_shape = (128,128,3))
conv_layer_2 = Conv2D(20,(3,3), activation='relu')
conv_layer_3 = Conv2D(10,(3,3), activation='relu')

#maxpooling layers
maxpool_layer_1 = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid', data_format=None)

#flattening layer
flatening_layer = Flatten()

#dense layers
dense_layer_1 = Dense(20, activation='relu')
dense_layer_2 = Dense(2, activation='softmax')

#model architecture initialization
model_layer_list = [conv_layer_1, conv_layer_2, conv_layer_3,
                    maxpool_layer_1, 
                    flatening_layer,
                    dense_layer_1, dense_layer_2]

model = Sequential(model_layer_list)
#model architecture overview
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 122, 122, 20)      2960      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 120, 120, 20)      3620      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 118, 118, 10)      1810      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 39, 39, 10)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 15210)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                304220    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 42        
Total para

In [15]:
#convolutional layers
conv_layer_1 = Conv2D(20,(7,7), activation='relu', input_shape = (128,128,3))
conv_layer_2 = Conv2D(20,(3,3), activation='relu')
conv_layer_3 = Conv2D(10,(3,3), activation='relu')

#maxpooling layers
maxpool_layer_1 = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid', data_format=None)

#flattening layer
flatening_layer = Flatten()

#dense layers
dense_layer_1 = Dense(20, activation='relu')
dense_layer_2 = Dense(2, activation='softmax')

#model architecture initialization
model_layer_list = [conv_layer_1, conv_layer_2, conv_layer_3,
                    maxpool_layer_1, 
                    flatening_layer,
                    dense_layer_1, dense_layer_2]

model = Sequential(model_layer_list)

#model optimizer initialisation
#model.compile(Adam(lr=0.0005),loss='categorical_crossentropy',metrics=['accuracy'])

#model architecture overview
#model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 122, 122, 20)      2960      
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 120, 120, 20)      3620      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 118, 118, 10)      1810      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 39, 39, 10)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 15210)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 20)                304220    
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 42        
Total para

Train your model using the `fit_generator` method and the two data generators you created earlier. Train for a reasonable amount of epochs, so as to get a good sense of how well this architecture performs.

Tips:
- Usually the bottleneck is when loading the images from the disk. To speed up training, make sure to take a look at the arguments `workers` and `use_multiprocessing` of `fit_generator`.
- You don't have to set the argument `steps_per_epoch` to the number of batches in an epoch. Instead, you can choose a lower number to obtain more frequent prints about the current loss and accuracy of your model (but then have in mind that you're not actually training for the number of epochs you specify in `epochs`).

In [7]:
# training model
history_graph = model.fit_generator(train_batches, steps_per_epoch = 20,
                    validation_data = validation_batches,
                    validation_steps = 10, epochs=50, verbose=2)

# saving history data to file
try:
    history = history.history
except AttributeError:
    print('History is already an dictionary')
    
path = "C:\\Users\\Ael\\Desktop\\deep-machine-learning-master2\\deep-machine-learning-master\\Home Assignments\\HA1\\HA1\\history_files"
            
list_files = next(os.walk(path))[2] 
name = 'History_graph_simle ' +  str(len(list_files))
with open(path + name + '.pkl', 'wb') as f:
        pickle.dump(history, f)
f.close()

#saving models to file
model_path = r'C:\Users\Ael\Desktop\deep-machine-learning-master2\deep-machine-learning-master\Home Assignments\HA1\HA1\history_files\Simple' 
list_files = next(os.walk(path_model))[2] 
model_name = '\Keras model simple CNN ' +  str(len(list_files))
model.save(model_path + model_name)

RuntimeError: You must compile your model before using it.

In [23]:
# saving history data to file
try:
    history = history_graph.history
except AttributeError:
    print('History is already an dictionary')
    
path = r"C:\Users\Ael\Desktop\deep-machine-learning-master2\deep-machine-learning-master\Home Assignments\HA1\HA1\history_files"
            
list_files = next(os.walk(path))[2] 
name = 'History_graph_simle ' +  str(len(list_files))
with open(path + name + '.pkl', 'wb') as f:
        pickle.dump(history, f)
f.close()

#saving models to file
model_path = r"C:\Users\Ael\Desktop\deep-machine-learning-master2\deep-machine-learning-master\Home Assignments\HA1\HA1\history_files\Simple" 
list_files = next(os.walk(model_path))[2] 
model_name = '\Keras model simple CNN ' +  str(len(list_files))
model.save(model_path + model_name)

Create one figure with two axes. In one of them, plot the loss in the training and the validation datasets. In the other one, plot the accuracy in the training and validation datasets.

Hint:
- The `fit_generator` method returns a `history` object.

In [24]:
list_files = os.listdir(path)
name = '\\History_graph_simple' +  str(len(list_files))   
with open(path + name + '.pkl', 'rb') as f:
        history_graph = pickle.load(f)
f.close()
x = np.linspace(0,len(history_graph['loss']),len(history_graph['loss']))

plt.subplot(2,1,1)
plt.plot(x, history_graph['loss'])
plt.plot(x, history_graph['val_loss'])
plt.legend(['Training loss','Validation loss'])

plt.subplot(2,1,2)
plt.plot(x, history_graph['acc'])
plt.plot(x, history_graph['val_acc'])
plt.legend(['Training Accurasy','Validation Accurasy'])

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Ael\\Desktop\\deep-machine-learning-master2\\deep-machine-learning-master\\Home Assignments\\HA1\\HA1\\history_files\\History_graph_simple1.pkl'

Based on these, what would you suggest for improving your model? Why?

**Your answer:** (fill in here)

---
## 3. Improving your initial model

Improve your initial model according to you answer above. Write the new definition in the cell below and train it.

In [8]:
#config = tf.ConfigProto(
#        device_count = {'GPU': 0}
#    )
#sess = tf.Session(config=config)

In [ ]:


model_improved = Sequential()

#convolutional layers
# 1st Convolutional Layer
model_improved.add(Conv2D(64,(9,9), input_shape=(128,128,3), strides=(2,2), padding='valid', activation='relu'))
# Pooling 
model_improved.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
# Batch Normalisation
model_improved.add(BatchNormalization())

model_improved.add(Conv2D(96,(3,3), activation='relu'))
model_improved.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
model_improved.add(BatchNormalization())

model_improved.add(Conv2D(128,(3,3), activation='relu'))
model_improved.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
model_improved.add(BatchNormalization())

#flattening layer
model_improved.add(Flatten())

#dense layers
model_improved.add(Dense(164, activation='relu'))
model_improved.add(Dropout(0.3))
model_improved.add(BatchNormalization())
model_improved.add(Dense(2, activation='softmax'))

#model architecture overview
model_improved.summary()

#model optimizer initialisation
model_improved.compile(Adam(lr=0.0002),loss='categorical_crossentropy',metrics=['accuracy'])

improved2_history_graph = model_improved.fit_generator(train_batches, steps_per_epoch = 50,
                    validation_data = validation_batches,
                    validation_steps = 20, epochs=60, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 60, 60, 64)        15616     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 96)        55392     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 96)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 14, 96)        384       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 128)       110720    
__________

How does the model perform, compared to the initial model? Create one plot with the training accuracy and another with the validation accuracy of the two scenarios.

In [ ]:
# YOUR CODE HERE

Did your results improve? Explain why, or why not.

**Your answer:** (fill in here)

---
## 4. Obtaining the *best* model

Continue to improve your model architecture by comparing the value of the metrics you're interested in both the training and validation set. Try different ideas, and consider comparing them using tensorboard. When you're happy with one architecture, copy it in the cell below and train it here. Save the optimization history (i.e. the `history` object returned by the `fit_generator`). You'll use this later to compare your best model with the one using transfer learning.

**Note**: When trying different ideas, you'll end up with several different models. However, when submitting your solutions to ping-pong, the cell below must contain only the definition and training of *one* model. Remove all code related to the models that were not chosen.

In [ ]:
# YOUR CODE HERE

Create one figure with two axes. In one of them, plot the loss in the training and the validation datasets. In the other one, plot the accuracy in the training and validation datasets.

In [ ]:
# YOUR CODE HERE

[Save your model](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model) to disk as a HDF5 file (the architecture, weights and optimizer state). This is simply so you can use it again easily in the later parts of the notebook, without having to keep it in memory or re-training it. The actual `.h5` files you create are not relevant to your ping-pong submission.

In [ ]:
model.save('my_model.h5')

---
## 5. Transfer Learning

Now, instead of trying to come up with a good architecture for this task, we'll use the VGG16 architecture, but with the top layers removed (the fully connected + classification layers). We'll substitute them with a single fully connected layer, and a classification layer that makes sense for our problem.

However, this model has a very high capacity, and will probably suffer a lot from overfitting if we try to train it from scratch, using only our small subset of data. Instead, we'll start the optimization with the weights obtained after training VGG16 on the ImageNet dataset.

Start by loading the VGG16 model without the top layers, from the `applications` submodule from Keras. Make sure to also load the weights obtained from the ImageNet pretraining.

In [25]:
vgg16_original = VGG16(weights='imagenet', include_top=True)
vgg16_original.summary()

MemoryError: 

Create a new model with the layers you want to add on top of VGG. The kernels and bias in these layers should be initialized randomly.

In [ ]:
final_layer = vgg16_original.get_layer('fc2').output
output = Dense(2, activation = 'softmax', name='output')(final_layer)

vgg16_custom = Model(Input(shape=(128,128,3)), output)
vgg16_custom.summary()

Now add the new model on top of VGG.

Tip:
- The VGG model you loaded from the `applications` submodule is from the [`Model`](https://keras.io/models/model/) class, not the `Sequential` class, so it doesn't have some methods you're used to (like `add`, for instance). It might be helpful to read [this introduction to the Model class](https://keras.io/getting-started/functional-api-guide/).

In [ ]:
# YOUR CODE HERE

### 5.1 Using VGG features

Now we're almost ready to train the new model. However, since the top layers of this architecture are being initialized randomly, it's sometimes possible for them to generate large gradients that can wreck the pretraining of the bottom layers. To avoid this, freeze all the VGG layers in your architecture (i.e. signal to the optimizer that these should not be changed during optimization) by setting the `trainable` attribute of them to `False`.

In [ ]:
# YOUR CODE HERE

Create the callbacks (if any) you would like to use, compile the model and train it.

In [ ]:
# YOUR CODE HERE

Create one figure with two axes. In one of them, plot the loss in the training and the validation datasets. In the other one, plot the accuracy in the training and validation datasets.

In [ ]:
# YOUR CODE HERE

How does the model perform, compared to the model obtained in step 4? Create one plot with the training accuracy and another with the validation accuracy of the two scenarios.

In [ ]:
# YOUR CODE HERE

Compare these results. Which approach worked best, starting from scratch or doing transfer learning? Explain how you evaluated this.

**Your answer:** (fill in here)

What are the main differences between the ImageNet dataset and the Dogs vs Cats dataset we used?

**Your answer:** (fill in here)

Even though there are considerable differences between these datasets, why is it that transfer learning is still a good idea?

**Your answer:** (fill in here)

In which scenario would transfer learning be unsuitable?

**Your answer:** (fill in here)

Save the model to a HDF5 file.

In [ ]:
model.save('trans_learning_top_only.h5')

### 5.2 Fine-tuning

Now that we have a better starting point for the top layers, we can train the entire network. Unfreeze the bottom layers.

Tip:
- Always recompile your model after changing anything in it!

In [ ]:
model = load_model('trans_learning_top_only.h5')

# YOUR CODE HERE

Create the callbacks (if any) you would like to use for this training here, compile the model, and train it.

Tip:
- Even though we do have a decent starting point for the optimization, it's still possible that a bad hyper-parameter choice wrecks the preinitialization. Make sure to use a small learning rate for this step.

In [ ]:
# YOUR CODE HERE

How does the model perform, compared to the model trained with freezed layers? Create one plot with the training accuracy and another with the validation accuracy of the two scenarios.

In [ ]:
# YOUR CODE HERE

Did the model's performance improve? Why (why not)?

**Your answer:** (fill in here)

Save the model to a HDF5 file.

In [ ]:
model.save('trans_learning_full.h5')

### 5.3 Improving the top model (optional)

Improve the architecture for the layers you add on top of VGG16. Try different ideas, and consider comparing them using tensorboard. When you're happy with one architecture, copy it in the cell below and train it here.

In [ ]:
# YOUR CODE HERE

How does the model perform, compared to the model trained in step 5.2? Create one plot with the training accuracy and another with the validation accuracy of the two scenarios.

In [ ]:
# YOUR CODE HERE

Save the model to a HDF5 file.

In [ ]:
model.save('best_trans_learning.h5')

## 6. Final training

Now we'll train the model that achieved the best performance so far using the entire dataset.

**Note**: start the optimization with the weights you obtained training in the smaller subset, i.e. *not* from scratch.

First, create two new data generators, one for training samples and one for validation samples. This time, they'll load data from the folders for the entire dataset.

In [ ]:
model = load_model('trans_learning_full.h5')

# YOUR CODE HERE

Create the callbacks you would like to use and train your model. This optimization might take a long time, so TensorBoard is advised ;).

In [ ]:
# YOUR CODE HERE

How does the model perform now when trained on the entire dataset, compared to when only trained on the smaller subset of data? Create one plot with the training accuracy and another with the validation accuracy of the two scenarios.

In [ ]:
# YOUR CODE HERE

What can you conclude from these plots?

**Your answer:** (fill in here)

## 7. Evaluation on test set

Now we'll evaluate your final model, obtained in step 6, on the test set. As mentioned before, the samples in the test set are not labeled, so we can't compute any performance metrics ourselves. Instead, we'll create a .csv file containing the predictions for each sample, and submit it to Kaggle for evaluation.

Compute the predictions for all samples in the test set according to your best model, and save it in a .csv file with the format expected by the competition.

Tip:
- There is a sample_submission file available for download in the same place where you downloaded the data from. Take a look at it to better understand what is the expected format here.

Hints:
- The Python module `os` has a `listdir` function, which returns the filenames of all files in a given path.
- If you don't know how to create and write to files with Python, Google can help.
- Keras has a submodule called `preprocessing.image`, with some handy functions (for instance `load_img` and `img_to_array`)

In [ ]:
# YOUR CODE HERE

Now that you created your submission file, submit it to Kaggle for evaluation. The [old competition](https://www.kaggle.com/c/dogs-vs-cats) does not allow submissions any more, so submit your file to the [new one](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition). Kaggle evaluates your submission according to your log-loss score. Which score did you obtain?

**Your answer:** (fill in here)

What was the username you used for this submission?

**Your answer:** (fill in here)